# Анализ сервиса вопросов и ответов по программированию

Необходимо проанализировать базу данных сервиса вопросов и ответов о программировании.
Анализ буду проводить с помощью SQL-запросов.


In [1]:
# загружаю необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine 

In [4]:
engine = create_engine(connection_string) 

1.
Найти количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [5]:
query = '''
SELECT COUNT(id)
FROM stackoverflow.posts 
WHERE post_type_id = 1 AND (favorites_count >= 100 OR score > 300)
''' 

In [6]:
qwestins_cnt = pd.read_sql_query(query, con = engine) 

In [7]:
qwestins_cnt.head()

,count
0,1355


2.
Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлю до целого числа.

In [8]:
query = '''
WITH a AS
(SELECT COUNT(id) AS id_cnt,
       CAST(DATE_TRUNC('DAY', creation_date) AS date) AS day_creation
FROM stackoverflow.posts 
WHERE post_type_id = 1 
AND creation_date::DATE BETWEEN '2008-11-01' AND '2008-11-18'
GROUP BY CAST(DATE_TRUNC('DAY', creation_date) AS date))
SELECT ROUND(AVG(id_cnt))
FROM a
''' 

In [9]:
qwestins_avg = pd.read_sql_query(query, con = engine) 

In [10]:
qwestins_avg

,round
0,383.0


3.
Сколько пользователей получили значки сразу в день регистрации? Выведу количество уникальных пользователей.

In [11]:
query = '''
SELECT COUNT( DISTINCT u.id)
FROM stackoverflow.users u
JOIN stackoverflow.badges b ON u.id=b.user_id
WHERE CAST(u.creation_date AS date) = CAST(b.creation_date AS date)
'''

In [12]:
badges_users = pd.read_sql_query(query, con = engine) 

In [13]:
badges_users

,count
0,7047


4.
Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [14]:
query = '''
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.posts p
JOIN  stackoverflow.users u ON p.user_id = u.id
JOIN stackoverflow.votes v ON p.id = v.post_id
WHERE u.display_name = 'Joel Coehoorn'
AND v.id > 0
'''

In [15]:
posts_uniq = pd.read_sql_query(query, con = engine) 

In [16]:
posts_uniq

,count
0,12


5.
Выгружу все поля таблицы vote_types. Добавлю к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблицу отсортирую по полю id.

In [17]:
query = '''
SELECT *,
       ROW_NUMBER()OVER(ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id
'''

In [18]:
vote_types_rank = pd.read_sql_query(query, con = engine) 

In [19]:
vote_types_rank.head()

,id,name,rank
0,1,AcceptedByOriginator,15
1,2,UpMod,14
2,3,DownMod,13
3,4,Offensive,12
4,5,Favorite,11


6.
Отберу 10 пользователей, которые поставили больше всего голосов типа Close. Отображу таблицу из двух полей: идентификатор пользователя и количество голосов. Отсортирую данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [20]:
query = '''
SELECT DISTINCT u.id,
       COUNT(v.id)
FROM stackoverflow.users u
JOIN stackoverflow.votes v ON u.id = v.user_id
JOIN stackoverflow.vote_types vt ON v.vote_type_id = vt.id
WHERE vt.name = 'Close'
GROUP BY u.id
ORDER BY COUNT(v.id) DESC, u.id DESC
LIMIT 10
'''

In [21]:
users_voite_top = pd.read_sql_query(query, con = engine) 

In [22]:
users_voite_top

,id,count
0,20646,36
1,14728,36
2,27163,29
3,41158,24
4,24820,23
5,9345,23
6,3241,23
7,44330,20
8,38426,19
9,19074,19


7.
Отберу 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отображу несколько полей:
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвою одно и то же место в рейтинге.
Отсортирую записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [23]:
query = '''
SELECT u.id,
       COUNT(b.id),
       DENSE_RANK()OVER(ORDER BY COUNT(b.id) DESC) AS rank
FROM stackoverflow.users u
JOIN stackoverflow.badges b ON u.id=b.user_id
WHERE CAST(b.creation_date AS date) BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY u.id
ORDER BY COUNT(b.id) DESC, u.id
LIMIT 10
'''

In [24]:
users_z_top = pd.read_sql_query(query, con = engine) 

In [25]:
users_z_top

,id,count,rank
0,22656,149,1
1,34509,45,2
2,1288,40,3
3,5190,31,4
4,13913,30,5
5,893,28,6
6,10661,28,6
7,33213,25,7
8,12950,23,8
9,25222,20,9


8.
Сколько в среднем очков получает пост каждого пользователя?
Сформирую таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Посты без заголовка, а также те, что набрали ноль очков учитывать не буду.

In [26]:
query = '''
WITH a AS
(SELECT title, user_id, score
FROM stackoverflow.posts
WHERE score <> 0
AND title IS NOT NULL)
SELECT *, ROUND(AVG(score)OVER(PARTITION BY user_id))
FROM a
'''

In [27]:
score_avg = pd.read_sql_query(query, con = engine) 

In [28]:
score_avg.head()

,title,user_id,score,round
0,Diagnosing Deadlocks in SQL Server 2005,1,82,573.0
1,How do I calculate someone's age in C#?,1,1743,573.0
2,Why doesn't IE7 copy <pre><code> blocks to the...,1,37,573.0
3,Calculate relative time in C#,1,1348,573.0
4,Wrapping StopWatch timing with a delegate or l...,1,92,573.0


9.
Отображу заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков исключу из списка.

In [29]:
query = '''
WITH a AS
(SELECT u.id,
       COUNT(b.id)
FROM stackoverflow.users u
JOIN stackoverflow.badges b ON u.id = b.user_id
group by u.id
HAVING COUNT(b.id) > 1000)
SELECT p.title
FROM stackoverflow.posts p
JOIN a ON p.user_id= a.id
where title IS NOT NULL
'''

In [30]:
posts_title = pd.read_sql_query(query, con = engine) 

In [31]:
posts_title

,title
0,What's the strangest corner case you've seen i...
1,What's the hardest or most misunderstood aspec...
2,What are the correct version numbers for C#?
3,Project management to go with GitHub


10.
Напишу запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделю пользователей на три группы в зависимости от количества просмотров их профилей:
- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отображу в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю не войдут в итоговую таблицу.

In [32]:
query = '''
SELECT id, views,
       CASE 
           WHEN views >= 350 THEN 1
           WHEN views >= 100 AND views < 350 THEN 2
           WHEN views < 100 THEN 3
       END
FROM stackoverflow.users
WHERE location LIKE '%%Canada%%'
AND views > 0
'''

In [33]:
users_canada = pd.read_sql_query(query, con = engine) 

In [34]:
users_canada

,id,views,case
0,22,1079,1
1,34,1707,1
2,37,757,1
3,41,174,2
4,42,1063,1
...,...,...,...
832,431315,103,2
833,266220,13,3
834,474548,24,3
835,455635,17,3


11.
Дополню предыдущий запрос. Отображу лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведу поля с идентификатором пользователя, группой и количеством просмотров. Отсортирую таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [35]:
query = '''
WITH
i AS (SELECT id, views,
CASE
WHEN views >= 350 THEN 1
WHEN 100 <= views AND views < 350 THEN 2
WHEN views < 100 THEN 3
END AS grouped
FROM stackoverflow.users
WHERE views > 0 AND location LIKE '%%Canada%%'),

l AS (SELECT id, views, grouped, MAX(views) OVER(PARTITION BY grouped) AS maximum
FROM i)

SELECT id, grouped, views
FROM l
WHERE views = maximum
ORDER BY views DESC, id
'''

In [36]:
users_canada_top = pd.read_sql_query(query, con = engine) 

In [37]:
users_canada_top

,id,grouped,views
0,3153,1,21991
1,46981,2,349
2,3444,3,99
3,22273,3,99
4,190298,3,99


12.
Посчитаю ежедневный прирост новых пользователей в ноябре 2008 года. Сформирую таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [38]:
query = '''
SELECT EXTRACT(DAY FROM creation_date::date) AS day,
       COUNT(id) AS users_cnt,
       SUM(COUNT(id))OVER(order BY EXTRACT(DAY FROM creation_date::date)) AS users_sum
FROM stackoverflow.users
WHERE creation_date::DATE BETWEEN '2008-11-01' AND '2008-11-30'
GROUP BY EXTRACT(DAY FROM creation_date::date)
'''

In [39]:
users_day_cnt = pd.read_sql_query(query, con = engine) 

In [40]:
users_day_cnt.head()

,day,users_cnt,users_sum
0,1.0,34,34.0
1,2.0,48,82.0
2,3.0,75,157.0
3,4.0,192,349.0
4,5.0,122,471.0


13.
Для каждого пользователя, который написал хотя бы один пост, найду интервал между регистрацией и временем создания первого поста. Отображу:
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [41]:
query = '''
with f AS
(SELECT user_id, creation_date, ROW_NUMBER () OVER(PARTITION BY USER_ID ORDER BY creation_date) AS fd
FROM stackoverflow.posts)       
SELECT u.id,
       f.creation_date - u.creation_date AS cnt_day
FROM stackoverflow.users u
JOIN f ON u.id=f.user_id
where f.fd = 1
'''

In [42]:
users_reg = pd.read_sql_query(query, con = engine) 

In [43]:
users_reg.head()

,id,cnt_day
0,1,0 days 09:18:29
1,2,0 days 14:37:03
2,3,3 days 16:17:09
3,4,15 days 05:44:22
4,5,1 days 14:57:51


14.
Выведу общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц пропускаю. Результат отсортирую по убыванию общего количества просмотров.

In [44]:
query = '''
SELECT DATE_TRUNC('month', creation_date)::date AS month,
       SUM(views_count)  AS total_views
FROM stackoverflow.posts
GROUP BY DATE_TRUNC('month', creation_date)::date
ORDER BY total_views desc
'''

In [45]:
cny_posts_2008 = pd.read_sql_query(query, con = engine) 

In [46]:
cny_posts_2008

,month,total_views
0,2008-09-01,452928568
1,2008-10-01,365400138
2,2008-11-01,221759651
3,2008-12-01,197792841
4,2008-08-01,131367083
5,2008-07-01,669895


15.
Выведу имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, учитывать не буду. Для каждого имени пользователя выведу количество уникальных значений user_id. Отсортирую результат по полю с именами в лексикографическом порядке.

In [47]:
query = ''' 
SELECT u.display_name,
       COUNT(DISTINCT p.user_id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u 
ON p.user_id=u.id
JOIN stackoverflow.post_types AS pt
ON pt.id=p.post_type_id
WHERE p.creation_date::date BETWEEN u.creation_date::date AND (u.creation_date::date + INTERVAL '1 month') 
      AND pt.type LIKE 'Answer'
GROUP BY u.display_name
HAVING COUNT(p.id) > 100
ORDER BY u.display_name;
'''

In [48]:
user_name_active = pd.read_sql_query(query, con = engine) 

In [49]:
user_name_active

,display_name,count
0,1800 INFORMATION,1
1,Adam Bellaire,1
2,Adam Davis,1
3,Adam Liss,1
4,Alan,8
...,...,...
74,lomaxx,1
75,mattlant,1
76,paxdiablo,1
77,tvanfosson,1


16.
Выведу количество постов за 2008 год по месяцам. Отберу посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортирую таблицу по значению месяца по убыванию.

In [50]:
query = '''
WITH users AS
(SELECT u.id
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u 
ON p.user_id=u.id
WHERE (u.creation_date::date BETWEEN '2008-09-01' AND '2008-09-30')
AND (p.creation_date::date BETWEEN '2008-12-01' AND '2008-12-31')
GROUP BY u.id)

SELECT DATE_TRUNC('month', p.creation_date)::date AS month,
       COUNT(p.id)
FROM stackoverflow.posts AS p
JOIN users ON users.id=p.user_id
GROUP BY DATE_TRUNC('month', p.creation_date)::date
ORDER BY month DESC
'''

In [51]:
cnt_posts_of_month = pd.read_sql_query(query, con = engine)

In [52]:
cnt_posts_of_month

,month,count
0,2008-12-01,17641
1,2008-11-01,18294
2,2008-10-01,27171
3,2008-09-01,24870
4,2008-08-01,32


17.
Используя данные о постах, выведу несколько полей:
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумма просмотров постов автора с накоплением.

Данные в таблице отсортирую по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [53]:
query = '''
SELECT user_id, creation_date, views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts
ORDER BY user_id, creation_date
'''

In [54]:
posts_df =  pd.read_sql_query(query, con = engine)

In [55]:
posts_df

,user_id,creation_date,views_count,sum
0,1,2008-07-31 23:41:00,480476,480476
1,1,2008-07-31 23:55:38,136033,616509
2,1,2008-07-31 23:56:41,0,616509
3,1,2008-08-04 02:45:08,0,616509
4,1,2008-08-04 04:31:03,0,616509
...,...,...,...,...
243791,5696608,2008-12-23 16:00:37,0,2804
243792,5696608,2008-12-23 17:35:09,0,2804
243793,5696608,2008-12-24 01:02:48,0,2804
243794,5696608,2008-12-30 14:34:45,0,2804


18.
Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберу дни, в которые он или она опубликовали хотя бы один пост, результат округлю до целого числа.

In [56]:
query = '''
WITH a AS
(SELECT user_id, 
       DATE_TRUNC('DAY', creation_date)::date AS days,
       COUNT(user_id) AS post_cnt
FROM stackoverflow.posts
WHERE creation_date BETWEEN '2008-12-01' AND '2008-12-07'
GROUP BY user_id, DATE_TRUNC('DAY', creation_date)::date),
b AS 
(SELECT user_id,
       COUNT(days) AS days_cnt
FROM a
GROUP BY user_id)
SELECT ROUND(AVG(days_cnt))
FROM b
'''

In [57]:
activity = pd.read_sql_query(query, con = engine)

In [58]:
activity

,round
0,2.0


19.
На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отображу таблицу со следующими полями:
Номер месяца.
Количество постов за месяц.
Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
Если постов стало меньше, значение процента будет отрицательным, если больше — положительным. Округлю значение процента до двух знаков после запятой.

In [59]:
query = '''
WITH a AS
(SELECT EXTRACT(MONTH FROM creation_date) AS month,
       COUNT(id) AS posts_cnt,
       LAG(COUNT(id)) OVER (ORDER BY EXTRACT(MONTH FROM creation_date))
FROM stackoverflow.posts
WHERE creation_date BETWEEN '2008-09-01' AND '2008-12-31'
GROUP BY EXTRACT(MONTH FROM creation_date))
SELECT month, posts_cnt,
       ROUND((posts_cnt::numeric/LAG(posts_cnt) OVER()-1)*100, 2)
FROM a
ORDER BY month
'''

In [60]:
posts_time = pd.read_sql_query(query, con = engine)

In [61]:
posts_time

,month,posts_cnt,round
0,9.0,70371,NaN
1,10.0,63102,-10.33
2,11.0,46975,-25.56
3,12.0,44592,-5.07


20.
Найду пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведу данные его активности за октябрь 2008 года в таком виде:
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [62]:
query = '''
WITH u AS
(SELECT user_id,
       COUNT(id) AS posts_cnt
FROM stackoverflow.posts
GROUP BY user_id
order by posts_cnt DESC
LIMIT 1)
SELECT EXTRACT(WEEK FROM p.creation_date) AS week,
       MAX(p.creation_date) 
FROM u JOIN stackoverflow.posts p ON u.user_id=p.user_id
WHERE DATE_TRUNC('month', p.creation_date)::date = '2008-10-01'
GROUP BY week;
'''

In [63]:
user_active_max = pd.read_sql_query(query, con = engine)

In [64]:
user_active_max

,week,max
0,40.0,2008-10-05 09:00:58
1,41.0,2008-10-12 21:22:23
2,42.0,2008-10-19 06:49:30
3,43.0,2008-10-26 21:44:36
4,44.0,2008-10-31 22:16:01
